## Use BERT to improve existing translations

In [5]:
!pip install --no-index tensorflow-gpu

Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
ERROR: tensorboard 2.1.0 has requirement setuptools>=41.0.0, but you'll have setuptools 40.8.0 which is incompatible.


In [2]:
!pip install --no-index /home/guest139/s3transfer/botocore
!pip install --no-index /home/guest139/s3transfer
!pip install --no-index /home/guest139/sacremoses
!pip install --no-index /home/guest139/transformers

Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /home/guest139/s3transfer/botocore
  Stored in directory: /tmp/pip-ephem-wheel-cache-27y_f2qy/wheels/fe/ab/37/f8ebb3c769c11a806041a139b0ab3964015975a5bb08071dc6
Successfully built botocore
  Found existing installation: botocore 1.15.37
    Uninstalling botocore-1.15.37:
      Successfully uninstalled botocore-1.15.37
Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /home/guest139/s3transfer
  Stored in directory: /tmp/pip-ephem-wheel-cache-fsq6w3nv/wheels/21/b5/7f/8dc1ac8a07cf30a6f8e4c8c6af478ed702be97967619f38b8b
Successfully built s3transfer
  Found existing installation: s3transfer 0.3.3
  

In [6]:
import tensorflow as tf

### Initialize

In [13]:
import sys
sys.path.insert(1, '../code')
import pretrained_tokenizer

t_fr = pretrained_tokenizer.Tokenizer(language='fr', path='../tokenizer_data_fr_30k')

I0413 14:32:19.699036 47207483892736 configuration_utils.py:281] loading configuration file ../tokenizer_data_fr_30k/config.json
I0413 14:32:19.700877 47207483892736 configuration_utils.py:319] Model config RobertaConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 0,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": 2,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "roberta",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 12,
  

In [14]:
from transformers import TFBertForMaskedLM, BertConfig
class bert_with_mask(tf.keras.Model):
    def __init__(self, config, onehot_mask):
        super(bert_with_mask, self).__init__()
        self.bert = TFBertForMaskedLM(config)
        self.onehot_mask = onehot_mask

    def call(self, inputs):
        mask = inputs[-1] # unpack mask from inputs
        inputs = inputs[:-1]
        outputs = self.bert(inputs)[0]
        
        outputs = tf.where(mask[:,:,None], outputs, self.onehot_mask[None,None,:])
        
        return (outputs,)

In [15]:
import numpy as np
onehot_mask = np.zeros(len(t_fr.tokenizer), dtype=np.float32)
onehot_mask[t_fr.tokenizer.pad_token_id] = 1.0

## Load model

In [16]:
# Recreate the model
config = BertConfig.from_pretrained('../code/bert_config_tiny.json')
config.vocab_size = len(t_fr.tokenizer)

new_model = bert_with_mask(config, onehot_mask)
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-3, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
new_model.compile(optimizer=optimizer, loss=loss)

I0413 14:32:21.458865 47207483892736 configuration_utils.py:281] loading configuration file ../code/bert_config_tiny.json
I0413 14:32:21.460298 47207483892736 configuration_utils.py:319] Model config BertConfig {
  "_num_labels": 2,
  "architectures": null,
  "attention_probs_dropout_prob": 0.1,
  "bad_words_ids": null,
  "bos_token_id": null,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "eos_token_id": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "min_length": 0,
  "model_type": "bert",
  "no_repeat_ngram_size": 0,
  "num_attention_heads": 2,
  "num_beam

In [17]:
# This initializes the variables used by the optimizers,
# as well as any stateful metric variables
SENTENCE_LENGTH = t_fr.MAX_LENGTH
new_model.train_on_batch((tf.zeros(shape=(1,SENTENCE_LENGTH),dtype=tf.int32), tf.ones(shape=(1,SENTENCE_LENGTH),dtype=tf.bool), tf.ones(shape=(1,SENTENCE_LENGTH),dtype=tf.bool)), (tf.zeros(shape=(1,SENTENCE_LENGTH),dtype=tf.int32)))

/localscratch/guest139.358248.0/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
W0413 14:32:24.196261 47207483892736 optimizer_v2.py:1043] Gradients do not exist for variables ['bert_with_mask_1/tf_bert_for_masked_lm_1/bert/pooler/dense/kernel:0', 'bert_with_mask_1/tf_bert_for_masked_lm_1/bert/pooler/dense/bias:0'] when minimizing the loss.
/localscratch/guest139.358248.0/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
W0413 14:32:26.332130 47207483892736 optimizer_v2.py:1043] Gradients do not exist for variables ['bert_with_mas

10.166824

In [18]:
# Load the state of the old model
new_model.load_weights('ckpts/weights-improvement-20-8.78.hdf5')

## Predict probs with new model

In [62]:
!pwd
# fr_file1 = '../data/train.lang2.sub'
fr_file1 = '../data/test_harman/test_fr_predictions.txt'
# fr_file1 = '../data/test_harman/test_fr_predictions2.txt'
# fr_file1 = '../data/test_harman/test_fr_predictions3.txt'
# fr_file1 = '../data/test_harman/test_fr_predictions4.txt'

/home/guest139/Translation-Team08-IFT6759/notebooks


In [63]:
def tokenize_file(filename):
    with open(filename, encoding="utf-8") as f:
        tokens = []
        attention_mask = []
        for idx, line in enumerate(f):
            encoded = t_fr.encode(line)
            tokens += [np.array(encoded['input_ids'])[None,:]]
            attention_mask += [np.array(encoded['attention_mask'])[None,:]]
    return np.concatenate(tokens,axis=0), np.concatenate(attention_mask,axis=0)

x_true, attention_mask = tokenize_file(fr_file1)
attention_mask = attention_mask.astype(bool)

In [64]:
pred = new_model((x_true[0,None], attention_mask[0,None], tf.ones(shape=(1,SENTENCE_LENGTH),dtype=tf.bool)))
pred[0]

<tf.Tensor: shape=(1, 64, 30000), dtype=float32, numpy=
array([[[-2.5675648e+01,  9.5021229e+00, -1.5938702e+01, ...,
         -2.6073385e+01, -2.5940208e+01, -2.2952614e+01],
        [-2.6017750e+01, -7.3445654e+00, -1.4740413e+01, ...,
         -2.5291624e+01, -2.4206230e+01, -2.2025677e+01],
        [-2.1415951e+01, -8.9850197e+00, -1.4591470e+01, ...,
         -2.2855679e+01, -2.4495766e+01, -1.7723272e+01],
        ...,
        [-1.7026737e+01, -1.7658922e+01, -2.5757554e+00, ...,
         -1.4725203e+01, -1.5865658e+01, -1.8158695e+01],
        [-1.4984193e+01, -1.3680983e+01, -4.9321651e-03, ...,
         -1.5642763e+01, -1.6222641e+01, -1.7623980e+01],
        [-1.6912313e+01, -9.8356638e+00,  1.6491385e+01, ...,
         -2.0540480e+01, -1.8524321e+01, -2.1160803e+01]]], dtype=float32)>

In [65]:
amax_pred = tf.squeeze(tf.argmax(pred[0], axis=2))
amax_pred

<tf.Tensor: shape=(64,), dtype=int64, numpy=
array([    1,   225,    36,   269,    11,   326,  1045,   317,   363,
         287,    11,   613,   360,   302,   618, 11952,   319,   289,
        1374,   276,     2,     2,     2,     2,     2,     2,     2,
           2,     2,     2,     2,     2,     2,     2,     2,     2,
           2,   360,   276,     2,     2,     2,     2,     2,     2,
         556,   276,     2,     2,     2,     2,     2,     2,   324,
           2,     2,    11,   359,   359,   360,   289,   311,     2,
           2])>

In [66]:
t_fr.decode(amax_pred[attention_mask[0]])

"C' est ainsi que nous n' avons pas et même imaginons la coopération ."

In [67]:
def correct_line(x, atn):
    pred = new_model((x[None,:], atn[None,:], tf.ones(shape=(1,SENTENCE_LENGTH),dtype=tf.bool)))
    amax_pred = tf.squeeze(tf.argmax(pred[0], axis=2))
    out_line = t_fr.decode(amax_pred[atn])
    return out_line

## Batch predict

In [68]:
lines = []
for x, atn in zip(x_true, attention_mask):
    lines += [correct_line(x, atn)]

In [69]:
# out_file =  '../data/train.lang2.out'
out_file = '../data/test_harman/test_fr_predictions.out'
# out_file = '../data/test_harman/test_fr_predictions2.out'
# out_file = '../data/test_harman/test_fr_predictions3.out'
# out_file = '../data/test_harman/test_fr_predictions4.out'

with open(out_file, 'w') as f:
    for line in lines:
        f.write(line + '\n')

## Compare the two files

In [57]:
!pip install --no-index /home/guest139/portalocker
!pip install --no-index /home/guest139/sacrebleu

Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /home/guest139/portalocker
  Stored in directory: /tmp/pip-ephem-wheel-cache-a7mrzrqf/wheels/1a/2a/f3/d2865b87939e065fc1c0d65412a9822a82942b171fae4b8b4f
Successfully built portalocker
ERROR: sacrebleu 1.4.3 requires typing, which is not installed.
  Found existing installation: portalocker 1.7.0
    Uninstalling portalocker-1.7.0:
      Successfully uninstalled portalocker-1.7.0
Ignoring pip: markers 'python_version < "3"' don't match your environment
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/avx, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Processing /home/guest139/sacrebleu
  Stored in directory: /tmp/pip-ephem-wheel-cache-irllmx5h/wheels/00/46/0a/0f56decffe5daa5de7d4c52af133903ed97462c76fb19064ec
Successfully built 

In [58]:
!pip uninstall --yes typing

Uninstalling typing-3.6.6:
  Successfully uninstalled typing-3.6.6


In [59]:
!sacrebleu

sacreBLEU: I need either a predefined test set (-t) or a list of references
sacreBLEU: The available test sets are:
           wmt19/dev: Development data for tasks new to 2019.
               wmt19: Official evaluation data.
       wmt18/test-ts: Official evaluation sources with extra test sets interleaved.
           wmt18/dev: Development data (Estonian<>English).
               wmt18: Official evaluation data.
       wmt17/tworefs: Systems with two references.
            wmt17/ms: Additional Chinese-English references from Microsoft Research.
      wmt17/improved: Improved zh-en and en-zh translations.
           wmt17/dev: Development sets released for new languages in 2017.
             wmt17/B: Additional reference for EN-FI and FI-EN.
               wmt17: Official evaluation data.
       wmt16/tworefs: EN-FI with two references.
           wmt16/dev: Development sets released for new languages in 2016.
             wmt16/B: Additional reference for EN-FI.
               wmt16

In [72]:
from evaluator import compute_bleu
print('Baseline')
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions.txt', print_all_scores=False)
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions2.txt', print_all_scores=False)
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions3.txt', print_all_scores=False)
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions4.txt', print_all_scores=False)

Baseline
final avg bleu score: 7.73
final avg bleu score: 9.87
final avg bleu score: 9.78
final avg bleu score: 10.63


In [73]:
print('With Masked LM:')
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions.out', print_all_scores=False)
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions2.out', print_all_scores=False)
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions3.out', print_all_scores=False)
compute_bleu('../data/test_harman/test_fr.txt', '../data/test_harman/test_fr_predictions4.out', print_all_scores=False)

With Masked LM:
final avg bleu score: 7.75
final avg bleu score: 9.85
final avg bleu score: 9.74
final avg bleu score: 10.63


In [32]:
lines[0]

"C' est mon village que nous et nous pouvons faire ce point de coopération avec l' espace ."

In [33]:
out_file

'../data/test_harman/test_fr_predictions3.out'